In [5]:
import torch
from diffusers import FluxPipeline
import diffusers
from PIL import Image
import matplotlib.pyplot as plt

In [9]:
_flux_rope = diffusers.models.transformers.transformer_flux.rope

def new_flux_rope(pos: torch.Tensor, dim: int, theta: int) -> torch.Tensor:
    assert dim % 2 == 0, "The dimension must be even."
    if pos.device.type == "cuda":
        return _flux_rope(pos.to("cpu"), dim, theta).to(device=pos.device)
    else:
        return _flux_rope(pos, dim, theta)
    diffusers.models.transformers.transformer_flux.rope = new_flux_rope·

AttributeError: module 'diffusers.models.transformers.transformer_flux' has no attribute 'rope'

In [11]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

# 1. 데이터 준비 (간단한 그래프 데이터)
# 노드 특징 (4개의 노드, 각 노드는 3차원 특징을 가짐)
x = torch.tensor([[1, 0, 1], [0, 1, 1], [1, 1, 0], [0, 0, 1]], dtype=torch.float)

# 엣지 리스트 (그래프 연결 관계)
edge_index = torch.tensor([
    [2, 2, 2, 3, 2, 1],  # From (시작 노드)
    [1, 0, 3, 2, 2, 3]   # To (끝 노드)
], dtype=torch.long)

# 노드 레이블 (0: Class 0, 1: Class 1)
y = torch.tensor([0, 1, 0, 1], dtype=torch.long)

# 그래프 데이터 생성
data = Data(x=x, edge_index=edge_index, y=y)

# 2. GCN 모델 정의
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)  # 첫 번째 GCN 레이어
        self.conv2 = GCNConv(hidden_dim, output_dim)  # 두 번째 GCN 레이어

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)  # 첫 번째 레이어 적용
        x = F.relu(x)  # 활성화 함수
        x = self.conv2(x, edge_index)  # 두 번째 레이어 적용
        return F.log_softmax(x, dim=1)  # 클래스 확률 출력

# 3. 모델 초기화
input_dim = data.x.shape[1]  # 입력 특징 차원 (3)
hidden_dim = 4               # 은닉층 차원
output_dim = 2               # 출력 클래스 수 (2)
model = GCN(input_dim, hidden_dim, output_dim)

# 4. 학습 설정
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

# 5. 모델 학습
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    out = model(data)  # 모델 예측
    loss = criterion(out, data.y)  # 손실 계산
    loss.backward()  # 역전파
    optimizer.step()  # 가중치 업데이트

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# 6. 평가
model.eval()
_, pred = model(data).max(dim=1)  # 예측 클래스
correct = int((pred == data.y).sum())  # 정확히 예측한 노드 수
accuracy = correct / data.num_nodes  # 정확도 계산
print(f"Accuracy: {accuracy:.4f}")


Epoch 0, Loss: 0.6837332248687744
Epoch 10, Loss: 0.6276774406433105
Epoch 20, Loss: 0.5794723033905029
Epoch 30, Loss: 0.5356159806251526
Epoch 40, Loss: 0.48747292160987854
Epoch 50, Loss: 0.4391833543777466
Epoch 60, Loss: 0.39375951886177063
Epoch 70, Loss: 0.35300424695014954
Epoch 80, Loss: 0.31652122735977173
Epoch 90, Loss: 0.28362613916397095
Accuracy: 1.0000
